# Init

In [ ]:
import os
from itertools import product
import re

import numpy as np
import pandas as pd

import matplotlib.pylab as plt
import seaborn as sns

from tqdm import tqdm

from luescher_nd.zeta.zeta3d import DispersionZeta3d
from luescher_nd.zeta.extern.pyzeta import zeta

from luescher_nd.hamiltonians.longrange import p_cot_delta

from luescher_nd.database.connection import DatabaseSession
from luescher_nd.database.tables import LongRangeEnergyEntry

from luescher_nd.hamiltonians.longrange import HBARC, M0, MN, GBAR0, E0, GAMMA0

MU0 = MN / 2

In [ ]:
%load_ext blackcellmagic

## Init Data

In [ ]:
DATA = os.path.join(os.getcwd(), os.pardir, "data")

DB = {}

DB["iv-c-fitted"] = os.path.abspath(os.path.join(DATA, "db-lr-iv-c-fitted.sqlite"))
DB["iv-c-fixed"] = os.path.abspath(os.path.join(DATA, "db-lr-iv-c-fixed.sqlite"))
DB["fv-d-fitted"] = os.path.abspath(os.path.join(DATA, "db-lr-fv-d-fitted.sqlite"))
DB["fv-c-fitted"] = os.path.abspath(os.path.join(DATA, "db-lr-fv-c-fitted.sqlite"))
DB["fv-d-fitted-10M0"] = os.path.abspath(os.path.join(DATA, "db-lr-fv-d-fitted-10M0.sqlite"))

dfs = []

for key, path in DB.items():
    sess = DatabaseSession(path, commit=False)
    df = pd.read_sql_table("long-range-energy", sess.engine, index_col="id")
    df["type"] = key
    df["L"] = df["n1d"] * df["epsilon"]
    df["E [MeV]"] = df["E"] * HBARC
    df["p2"] = df["E"] * 2 * MU0
    df["nstep"] = df["nstep"].fillna(-1).astype(int)
    df["x"] = df["p2"] / (2 * np.pi / df["L"])**2
    df = df.dropna()
    
    dfs.append(df)

DF = pd.concat(dfs, ignore_index=True)
DF.head()

In [ ]:
lattice_pars = ["L", "epsilon", "nstep"]
cols = [col for col in DF.columns if col not in ["dispersion", "spherical"]]

lattice_zetas = {}

for L, n1d, nstep in tqdm(list(product(DF.L.unique(), DF.n1d.unique(), DF.nstep.unique()))):
    epsilon = L / n1d
    ind = (DF.L == L) & (DF.n1d == n1d) & (DF.nstep == nstep)
    z = DispersionZeta3d(L, epsilon, nstep if nstep > 0 else None)
    DF.loc[ind, "dispersion"] = z(DF.loc[ind, "x"].values) / np.pi / DF.loc[ind, "L"]

DF.loc[DF.nstep == -1, "nstep"] = "$\infty$"
DF["nstep"] = DF["nstep"].astype(str)
    
print("-> Now computing spherical Lüscher results (also takes some time)")
DF["spherical"] = zeta(DF.x.values) / np.pi / DF.L.values

DF = pd.melt(DF, id_vars=cols, var_name="luescher", value_name="y")

DF.head()

## Helper functions

In [ ]:
def plot_ere(*args, **kwargs):
    """Plots effective range expansion for data and analytic result.
    """
    ax = plt.gca()
    p2range = np.linspace(args[0].min(), args[0].max(), 100)
    
    ax.plot(*args, **kwargs)
    ax.plot(
        p2range,
        p_cot_delta(np.sqrt(p2range+0j), GBAR0, MU0, M0).real,
        label="Analytic",
        ls="-",
        color="black",
        zorder=-1,
        lw=1,
    )

# Plots

In [ ]:
print("options for type:", DF.type.unique())
print("options for nstep:", DF.nstep.unique())

What correspond to what:
* `iv-c-fixed`: Parameters for finite volume lattice hamiltonian where chosen to be the same as in infinite volume continuum
* `iv-c-fitted`: Parameters for finite volume lattice hamiltonian where fitted to reproduce infinite volume continuum boundstate
* `fv-c-fitted`: Parameters for finite volume lattice hamiltonian where fitted to intersection of standard Lüscher intersection with analytic effective range expansion 
* `fv-d-fitted`: Parameters for finite volume lattice hamiltonian where fitted to intersection of dispersion Lüscher intersection with analytic effective range expansion 
* `fv-d-fitted-10M0`: Same as above with potential looking more like a delta function ($M = 10 M_0$).

Change the query kwargs to produce different fits (run the next 3 cells)

In [ ]:
query_kwargs = {
    "type": "'iv-c-fitted'", # note the double quotations
    "nstep": "'$\infty$'"
}
col = "$L$ [fm$^{-1}$]"
row = "n1d"
hue = "luescher"
x = "$p^2$ [fm$^{-2}$]"
y = "$p \cot(\delta_0(p))$ [fm$^{-1}$]"

In [ ]:
query = " and ".join(f"{key} == {val}" for key, val in query_kwargs.items())
query += " and x > -5 and y < 2 and y > -2 and nstep != '1'"
query += " and x > -5 and nstep != '1'"
query += " and date > '2019-05-4 06:00:00.000' and date < '2019-05-05 15:00:00'"

df = DF.query(query).sort_values("x")
for key, val in {
    "y": "$p \cot(\delta_0(p))$ [fm$^{-1}$]",
    "L": "$L$ [fm$^{-1}$]",
    "p2": "$p^2$ [fm$^{-2}$]",
}.items():
    df[val] = df[key]

df = df[df.n1d % 10 == 0]


In [ ]:
grid = sns.FacetGrid(
    data=df,
    col=col,
    hue=hue,
    row=row,
    sharex=True,
    sharey=True,
    legend_out=True,
    hue_order={"spherical", "dispersion"},
)

grid.map(plot_ere, x, y, ms=1, lw=0.5, ls="--", marker="s")

grid.fig.set_dpi(200)
grid.fig.suptitle(", ".join(f"{key} = {val}" for key, val in query_kwargs.items()), y=1.02)
grid.add_legend(frameon=False)

plt.show()